#### NOTE! This algorithm is not yet finished

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Load the dataset 
load diabetes_data

In [32]:
diabetes_df = pd.read_csv('diabetes_data.csv')

## FEATURE SELECTION

1. A function which drops columns if they have zeros more than some threshold

In [33]:
def drop_cols_with_zeros(df, threshold=0.9):
    new_df = df.replace({0:np.nan})
    new_df.dropna(thresh=0.9*len(diabetes_df), axis='columns', inplace=True)    
    return new_df

2. A function which replaces zeros in columns with means of those columns

In [34]:
def replace_zeros_with_means(df):
    for feature in df.columns:
        nonzero_mean = df.loc[X[feature]!=0, feature].mean()
        df.loc[df[feature]==0, feature] = nonzero_mean
        df[feature].fillna(nonzero_mean, inplace=True)
    return df

3. A function which finds columns whose correlation with target columns is greater than given threshold

In [35]:
def find_best_features(X,y, threshold=0.2):
    columns_to_drop = []
    for column in X.columns:
        if abs(np.corrcoef(X[column].values, y.values)[0,1])<threshold:
            columns_to_drop.append(column)
    return X[X.columns.difference(columns_to_drop)]


Applying feature selection to columns

In [36]:
target_col = 'Outcome'
y = diabetes_df[target_col]
X_original = diabetes_df[diabetes_df.columns.difference([target_col])]
X = drop_cols_with_zeros(X_original)
X = replace_zeros_with_means(X)
X = find_best_features(X, y)

## Divide the dataset into train and test datasets

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Implement the following functions to create your own logistic regression algorithm from scratch.
#### Feel free to use more additinal functions in your implementation.

## NOTE! Since data juggling between several function, especially in Jupyter Notebook, is a little bit annoying, I've incapsulated all in one class similar as in Scikit Learn

In [38]:
class SimpleLogisticRegression:

    def fit(self, X, Y, threshold=0.5, learning_rate=1e-4):
        # threshold is value by which outcome probabilities are binarizied
        self.threshold = threshold
        self.gradient_descent(X.values, Y.values, step_size=learning_rate)

    def predict(self, X):
        return pd.Series(np.where(self.sigmoid(X, self.beta) > self.threshold, 1, 0))

    def get_beta(self):
        return self.beta

    def sigmoid(self, X, beta):
        """
        :param X: data matrix (2 dimensional np.array)
        """
        dot = np.dot(X, beta)
        return 1 / (1 + np.exp(-dot))

    def gradient(self, beta, X, Y):
        """
        :param X: data matrix (2 dimensional np.array)
        :param Y: response variables (1 dimensional np.array)
        :param beta: value of beta (1 dimensional np.array)
        :return: np.array i.e. gradient according to the data
        """
        return np.dot(self.sigmoid(X, beta) - Y, X)

    def cost_func(self, X, Y, beta):
        """
        :param X: data matrix (2 dimensional np.array)
        :param Y: response variables (1 dimensional np.array)
        :param beta: value of beta (1 dimensional np.array)
        :return: numberic value of the cost function

        """
        n = len(X)
        hypoth = self.sigmoid(X, beta)
        cost = 1 / n * (np.dot(-Y, np.log(hypoth)) - np.dot( (1 - Y),  np.log(1 - hypoth)))
        return cost

    def gradient_descent(self, X, Y, epsilon=1e-6, step_size=1e-4, max_steps=10_000):
        """
        :param X: data matrix (2 dimensional np.array)
        :param Y: response variables (1 dimensional np.array)
        :param epsilon: threshold for a change in cost function value
        :param max_steps: maximum number of iterations before algorithm will terminate.
        """
        beta = np.zeros(X.shape[1] + 1)
        # adding columns with ones to X
        X = np.insert(X, 0, 1, axis=1)

        step = 1
        while step < max_steps:
            beta -= step_size * self.gradient(beta, X, Y)
            if np.abs(self.cost_func(X, Y, beta)) < epsilon:
                print('Gradient descent converged on {}-th step'.format(str(step)))
                break
            step += 1
        else:
            print('Can not converge gradient descent with given epsilon, step size and maximum steps')
            print('Last value of beta was ', str(beta))

        self.beta = beta

## Run logistic regression using the features of your choice and using "Outcome" as a target variable

In [39]:
my_clf = SimpleLogisticRegression()
my_clf.fit(X_train, y_train)

X_array = np.insert(X_test.values, 0, 1, axis=1)

my_y_pred = my_clf.predict(X_array)

print('Accuracy of my classifier: ', accuracy_score(y_test, my_y_pred))

/home/karen/anaconda3/envs/ml_beginner_course/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log
/home/karen/anaconda3/envs/ml_beginner_course/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


Can not converge gradient descent with given epsilon, step size and maximum steps
Last value of beta was  [-203.54551284   -0.46034571   -0.83588061    1.94984376]
Accuracy of my classifier:  0.6298701298701299


## Use the logistic regression available in Sklearn on the same data.

In [40]:
sklearn_clf = LogisticRegression()
sklearn_clf.fit(X_train, y_train)
y_pred = sklearn_clf.predict(X_test)
print('Accuracy of sklearn regressor: ', accuracy_score(y_test, y_pred))

Accuracy of sklearn regressor:  0.7337662337662337


## Extra: try to plot the results of your algorithm i.e. a scatter plot of points classified into 2 classes in different colors.
